In [1]:
%matplotlib widget
from backtrader import plot
import backtrader as bt
import backtrader.analyzers as btanalyzers
import rba_tools as rba
import pandas as pd
import numpy as np
#import matplotlib
from datetime import datetime,date,timedelta
from backtrader.utils import num2date,date2num
import rba_tools.backtest.rba_backtrader_set as rbs
import plotly.graph_objects as go

In [2]:
class MaCrossStrategy(bt.Strategy):

    def __init__(self):
        ma_fast = bt.ind.SMA(period = 10)
        ma_slow = bt.ind.SMA(period = 50)

        self.crossover = bt.ind.CrossOver(ma_fast, ma_slow)

    def next(self):
        if not self.position:
            if self.crossover > 0:
                self.buy()
        elif self.crossover < 0:
            self.close()

In [3]:
cerebro = bt.Cerebro()
puller = rba.retriever.get_crypto_data.DataPuller.kraken_puller()
symbol = 'ETH/USD'
from_date = '1/1/20'
to_date = '12/31/20'

In [4]:
dataframe = puller.fetch_df(symbol, 'h', from_date, to_date)
data = bt.feeds.PandasData(dataname=dataframe,
                               # datetime='Date',
                               nocase=True,
                               )
cerebro.adddata(data)
cerebro.addstrategy(MaCrossStrategy)
cerebro.broker.setcash(1000.0)
cerebro.addsizer(bt.sizers.PercentSizer, percents = 10)

In [5]:
back = cerebro.run()

In [6]:
strategy = back[0]
from backtrader import plot
plotter = plot.Plot()
plotter

In [7]:
plotter.sortdataindicators(strategy)

In [8]:
vars(plotter)

{'params': <backtrader.metabase.AutoInfoClass_Plot_OldSync at 0x2257f36cac0>,
 'p': <backtrader.metabase.AutoInfoClass_Plot_OldSync at 0x2257f36cac0>,
 'dplotstop': [<backtrader.observers.broker.Broker at 0x2257f4af130>,
 'dplotsup': defaultdict(list, {}),
 'dplotsdown': defaultdict(list,
             {<backtrader.indicators.sma.SMA at 0x225497873d0>: [<backtrader.indicators.crossover.CrossOver at 0x2257f47d0d0>]}),
 'dplotsover': defaultdict(list,
             {<backtrader.feeds.pandafeed.PandasData at 0x2254a525bb0>: [<backtrader.observers.buysell.BuySell at 0x2257f4af400>,
               <backtrader.indicators.sma.SMA at 0x2257f2e6880>]})}

In [9]:
plot_result = plotter.plot(strategy)
plot_result

<IPython.core.display.Javascript object>

[<Figure size 640x480 with 5 Axes>]

In [10]:
st_dtime = strategy.lines.datetime.plot()

In [11]:
print(plotter)

In [12]:
vars(plotter)

{'params': <backtrader.metabase.AutoInfoClass_Plot_OldSync at 0x2257f36cac0>,
 'p': <backtrader.metabase.AutoInfoClass_Plot_OldSync at 0x2257f36cac0>,
 'dplotstop': [<backtrader.observers.broker.Broker at 0x2257f4af130>,
 'dplotsup': defaultdict(list,
             {<backtrader.feeds.pandafeed.PandasData at 0x2254a525bb0>: [],
              <backtrader.observers.buysell.BuySell at 0x2257f4af400>: [],
              <backtrader.indicators.sma.SMA at 0x225497873d0>: [],
              <backtrader.indicators.sma.SMA at 0x2257f2e6880>: [],
              <backtrader.indicators.crossover.CrossOver at 0x2257f47d0d0>: []}),
 'dplotsdown': defaultdict(list,
             {<backtrader.indicators.sma.SMA at 0x225497873d0>: [<backtrader.indicators.crossover.CrossOver at 0x2257f47d0d0>],
              <backtrader.observers.buysell.BuySell at 0x2257f4af400>: [],
              <backtrader.indicators.crossover.CrossOver at 0x2257f47d0d0>: [],
              <backtrader.indicators.sma.SMA at 0x2257f2e6880>:

In [13]:
over = plotter.dplotsover
over

defaultdict(list,
            {<backtrader.feeds.pandafeed.PandasData at 0x2254a525bb0>: [<backtrader.observers.buysell.BuySell at 0x2257f4af400>,
             <backtrader.observers.broker.Broker at 0x2257f4af130>: [],
             <backtrader.observers.trades.Trades at 0x2257f4af6a0>: [],
             <backtrader.observers.buysell.BuySell at 0x2257f4af400>: [],
             <backtrader.indicators.sma.SMA at 0x225497873d0>: [],
             <backtrader.indicators.sma.SMA at 0x2257f2e6880>: [],
             <backtrader.indicators.crossover.CrossOver at 0x2257f47d0d0>: []})

In [14]:
type(plotter.dplotsup)

collections.defaultdict

In [15]:
mylist = [i for i in plotter.dplotsover]
mylist

In [16]:
ind = mylist[4]
indicator = ind

In [17]:
lineidx = 0
line = ind.lines[lineidx]
linealias = ind.lines._getlinealias(lineidx)

In [18]:
line

In [19]:
vals = line.plotrange(0,len(line))

In [23]:
np.arange(5)

array([0, 1, 2, 3, 4])

In [26]:
np.array(vals)

array([    nan,     nan,     nan, ..., 736.722, 737.312, 737.685])

In [27]:
fig = go.Figure(data=go.Scatter(x=np.arange(len(line)),y=np.array(vals)))
fig.show()

In [28]:
def testplotind(indicator: bt.indicator, x_axis):
    for lineidx in range(indicator.size()):
            line = indicator.lines[lineidx]
            linealias = indicator.lines._getlinealias(lineidx)

            lineplotinfo = getattr(indicator.plotlines, '_%d' % lineidx, None)
            if not lineplotinfo:
                lineplotinfo = getattr(indicator.plotlines, linealias, None)

            if not lineplotinfo:
                lineplotinfo = bt.AutoInfoClass()

            if lineplotinfo._get('_plotskip', False):
                continue


            # plot data
            indicator_vals = line.plotrange(0, len(line))

            return go.Scatter(x=x_axis,y=np.array(vals))

In [32]:
myp = testplotind(ind, np.array(st_dtime))
go.Figure(myp).show()

In [34]:
d = st_dtime[0]
d

737425.0

In [42]:
num2date(np.array(st_dtime))

TypeError: only size-1 arrays can be converted to Python scalars